<a href="https://colab.research.google.com/github/safwan0099/Generative-AI-first-try-/blob/main/Gen_AI_(Fine_tuning_%2B_optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig


In [3]:
ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# !git config --global user.email "safwanbinamir@gmail.com"
# !git config --global user.name "safwan0099"

In [5]:
# !git clone https://github.com/safwan0099/Generative-AI-first-try-.git

fatal: destination path 'Generative-AI-first-try-' already exists and is not an empty directory.


In [10]:
!cd Generative-AI-first-try-
!git add .


fatal: not a git repository (or any of the parent directories): .git


In [15]:
!cd ../../..
!ls

Generative-AI-first-try-  sample_data


In [12]:
!ls

Generative-AI-first-try-  sample_data
